# Purpose

This notebook takes images from the training set located at http://www.cvlibs.net/datasets/kitti/eval_object.php and combines the annotations with the images, effectively drawing bounding boxes on the images.

The purpose is to see what the data looks like before we start working with it.

In [10]:
import xml.etree.cElementTree as et
import os

class AnnotationFile:
    
    def __init__(self, path, out_path):
        self.annotation_count = 0
        self.path = path
        self.out_path = out_path
    
    def __enter__(self):
        root = et.Element('annotation', verified='yes')
        self.root = root
        folder = et.SubElement(root, 'folder')
        folder.text = 'images'
        filename = et.SubElement(root, 'filename')
        filename.text = os.path.basename(self.path)
        path = et.SubElement(root, 'path')
        path.text = self.path
        source = et.SubElement(root, 'source')
        database = et.SubElement(source, 'database')
        database.text = 'unknown'
        size = et.SubElement(root, 'size')
        width = et.SubElement(size, 'width')
        width.text = '1242'
        height = et.SubElement(size, 'height')
        height.text = '375'
        depth = et.SubElement(size, 'depth')
        depth.text = '3'
        segmented = et.SubElement(root, 'segmented')
        segmented.text = '0'
        return self
    
    def add_annotation(self, label, xmin, xmax, ymin, ymax):
        obj = et.SubElement(self.root, 'object')
        name = et.SubElement(obj, 'name')
        name.text = label
        pose = et.SubElement(obj, 'pose')
        pose.text = 'Unspecified'
        truncated = et.SubElement(obj, 'truncated')
        truncated.text = '0'
        difficult = et.SubElement(obj, 'difficult')
        difficult.text = '0'
        box = et.SubElement(obj, 'bndbox')
        x1 = et.SubElement(box, 'xmin')
        x1.text = str(xmin)
        x2 = et.SubElement(box, 'ymin')
        x2.text = str(ymin)
        x3 = et.SubElement(box, 'xmax')
        x3.text = str(xmax)
        x4 = et.SubElement(box, 'ymax')
        x4.text = str(ymax)
        self.annotation_count += 1

    def __exit__(self, exec_type, exec_val, exec_traceback):
        if self.annotation_count > 0:
            tree = et.ElementTree(self.root)
            tree.write(self.out_path, xml_declaration=False)

In [11]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import progressbar as pb
%matplotlib inline

# Given the OpenCV image object and dataframe of its training data, annotate with bounding box and label
def annotate_image(img, df):
    for row in df.iterrows():
        label = row[1][0]
        if label not in ['Car', 'Pedestrian']:
            continue
        bounds = np.round(row[1][4:8].tolist()).astype(int)
        cv2.rectangle(img, tuple(bounds[:2]), tuple(bounds[2:]), (0, 255, 0), 2)
        cv2.putText(img, label, (bounds[0], bounds[1] - 5), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
    return img

# Format integer to a string (images and labels are called 000015.png and 000015.txt)
def format_number(i):
    return '{:06d}'.format(i)

# Load traing data image and object label file converted into a dataframe
def load_data(i):
    s = format_number(i)
    img = cv2.imread('/home/ryan/Desktop/KITTI_Vision/data_object_image_2/training/image_2/{:s}.png'.format(s), 1)
    df = pd.read_csv('/home/ryan/Desktop/KITTI_Vision/training/label_2/{:s}.txt'.format(s), sep='\s+', header=None)
    return img, df

# Plot image if you'd like
def plot_image(img):
    plt.figure(figsize=[15,10])
    plt.imshow(img)
    plt.show()

In [119]:
# Go through all images and annotate them with boxes and text
bar = pb.ProgressBar()
for i in bar(range(7480)):
    img = annotate_image(*load_data(i))
    cv2.imwrite('/home/ryan/Desktop/KITTI_Vision/labeled/{:s}.png'.format(format_number(i)), img)

100% (7480 of 7480) |#####################| Elapsed Time: 0:05:52 Time: 0:05:52


In [16]:
# Write XML annotation files
bar = pb.ProgressBar()
for i in bar(range(7480)):
    img, df = load_data(i)
    path = '/home/ryan/Desktop/KITTI_Vision/data_object_image_2/training/image_2/{:06d}.png'.format(i)
    out_path = '/home/ryan/Desktop/KITTI_Vision/data_object_image_2/training/image_2_annots/{:06d}.xml'.format(i)
    with AnnotationFile(path, out_path) as af:
        for row in df.iterrows():
            label = row[1][0]
            if label not in ['Car']:
                continue
            bounds = np.round(row[1][4:8].tolist()).astype(int)
            af.add_annotation(label, bounds[0], bounds[2], bounds[1], bounds[3])

100% (7480 of 7480) |#####################| Elapsed Time: 0:02:52 Time: 0:02:52
